In [ ]:
import numpy as np
import dill
import pandas as pd

import datetime as dt
from tqdm import tqdm
from collections import defaultdict, Counter

import warnings

warnings.filterwarnings("ignore")

In [ ]:
III = pd.read_csv('ouput/Final_III_0.csv')
print(III.shape,len(III.HADM_ID.unique()),len(III.ICUSTAY_ID.unique()))
III['DIEINHOSPITAL'].value_counts()bb

In [ ]:
c_dict = {#included ts itemid and corresponding text
}
print(len(c_dict))

all_charts = [int(item.split('_')[-1]) for item in list(c_dict.keys())]
all_charts = list(pd.unique(all_charts))
len(all_charts)

In [ ]:
#CHARTEVENTS
dataset_path = 'input/CHARTEVENTS.csv.gz'

chunksize = 10000000
final=pd.DataFrame()
for chunk in tqdm(pd.read_csv(dataset_path, compression='gzip', header=0, index_col=None, chunksize=chunksize)):
    chunk = chunk[['ICUSTAY_ID', 'CHARTTIME', 'ITEMID', 'VALUE', 'VALUENUM']]
    chunk = chunk[chunk['ICUSTAY_ID'].isin(III['ICUSTAY_ID'])]
    chunk = chunk[chunk['ITEMID'].isin(all_charts)]
    chunk = chunk.dropna(subset=['VALUE', 'VALUENUM'], how='all')
    if chunk.shape[0] > 0:
        final = pd.concat([final, chunk], ignore_index=True)

In [ ]:
print(len(final.ICUSTAY_ID.unique()))
print(final[final.VALUENUM.isna()].shape)

In [ ]:
def is_numeric(value):
    try:
        float(value)
        return True
    except (ValueError, TypeError):
        return False

In [ ]:
final_match = pd.merge(final,III[['HADM_ID','ICUSTAY_ID','INTIME', 'OUTTIME']],on='ICUSTAY_ID',how='left')
print(final_match.shape)

final_match['INTIME'] = pd.to_datetime(final_match['INTIME'])
final_match['OUTTIME'] = pd.to_datetime(final_match['OUTTIME'])
final_match['CHARTTIME'] = pd.to_datetime(final_match['CHARTTIME'])

within_criteria = final_match['CHARTTIME'].between(final_match['INTIME'],final_match['OUTTIME'])
within = final_match[within_criteria]

within['event_time_from_admit'] = within['CHARTTIME'] - within['INTIME']
within['start_time']=within['event_time_from_admit'].dt.total_seconds() / 3600
within['los'] = (within['OUTTIME'] - within['INTIME']).dt.total_seconds() / 3600
within=within[within['start_time']>=0]
print(within.shape)

within_48 = within[within['start_time']<=48]
print(within_48.shape)

within_48['is_numeric'] = within_48['VALUENUM'].apply(is_numeric)

result = within_48.groupby('ITEMID')['is_numeric'].all().reset_index()
result.rename(columns={'is_numeric': 'is_fully_numeric'}, inplace=True)

num_col = result[result.is_fully_numeric==True].ITEMID.values
print(len(within_48.ITEMID.value_counts()),len(num_col))

within_48_make = within_48[['ICUSTAY_ID','ITEMID','VALUENUM']]
agg_df = within_48_make.groupby(['ICUSTAY_ID', 'ITEMID'])['VALUENUM'].agg(
    min='min', 
    max='max', 
    mean='mean', 
    median='median'
).reset_index()

print(agg_df.head())

agg_df =agg_df.drop_duplicates()
matching_columns = list(c_dict.keys())

agg_df['ITEMID'] = agg_df['ITEMID'].astype(str)
agg_df = agg_df[agg_df['ITEMID'].isin(matching_columns)]
agg_df['ITEMID'] = agg_df['ITEMID'].map(c_dict)
print(agg_df.head())

print(len(agg_df.ITEMID.unique()))
print(agg_df.ITEMID.value_counts())

pivot_df = agg_df.pivot_table(
    index='ICUSTAY_ID', 
    columns='ITEMID', 
    values=['min', 'max', 'mean', 'median'],
    aggfunc='first'  
)

pivot_df.columns = [f'{col[1]}_{col[0]}' for col in pivot_df.columns]
pivot_df = pivot_df.reset_index()

pivot_df = pivot_df.fillna(pivot_df.median())
print(pivot_df.columns.tolist())
pivot_df.head()

In [ ]:
def create_Dict(chart):
    all_gcs = pd.DataFrame()
    
    for hid in tqdm(chart.ICUSTAY_ID.unique()):  
        df2 = chart[chart['ICUSTAY_ID'] == hid]  
        
        if df2.shape[0] == 0:
            print(f'Empty：{hid}')

        else:
            val = df2.pivot(index='start_time', columns='ITEMID', values='VALUENUM')
            val['GCS'] = val.sum(axis=1)
            gcs_min = val['GCS'].min()
            gcs_max = val['GCS'].max()
            gcs_mean = val['GCS'].mean()
            gcs_median = val['GCS'].median()
            
            val = pd.DataFrame({
                'GCS_min': [gcs_min],
                'GCS_max': [gcs_max],
                'GCS_mean': [gcs_mean],
                'GCS_median': [gcs_median]
            })

            val['ICUSTAY_ID'] = int(hid)

            all_gcs = pd.concat([all_gcs,val])
    
    return all_gcs

    
final_chart=pd.DataFrame()
t=0

for i in tqdm(np.arange(0, 48, 0.25)): 
    sub_chart = within_48[(within_48['start_time'] >= i) & (within_48['start_time'] < i + 0.25)]
    sub_chart_num = sub_chart[sub_chart['ITEMID'].isin(num_col)].groupby(['ICUSTAY_ID', 'ITEMID']).agg({'VALUENUM': np.nanmean}).reset_index()
    sub_chart = sub_chart_num

    sub_chart['start_time'] = t
    
    if final_chart.empty:
        final_chart = sub_chart
    else:
        final_chart = final_chart.append(sub_chart)

    t += 1


final_chart = final_chart[final_chart.ITEMID.isin([220739,223901,223900])]
final_chart['ITEMID'] = final_chart['ITEMID'].astype(str)
final_chart['ITEMID'] = final_chart['ITEMID'].map(c_dict)
print(final_chart.ITEMID.value_counts())

all_gcs = create_Dict(final_chart)
pivot_df = pd.merge(pivot_df,all_gcs,on='ICUSTAY_ID',how='left')
pivot_df = pivot_df.fillna(pivot_df.median())
print(pivot_df.shape,len(pivot_df.ICUSTAY_ID.unique()))

In [ ]:
#LABEVENTS
dataset_path = 'input/LABEVENTS.csv.gz'

chunksize = 10000000
final=pd.DataFrame()
for chunk in tqdm(pd.read_csv(dataset_path, compression='gzip', header=0, index_col=None, chunksize=chunksize)):
    chunk.columns = chunk.columns.str.upper()
    chunk = chunk[['HADM_ID', 'CHARTTIME', 'ITEMID', 'VALUE', 'VALUENUM']]

    chunk = chunk[chunk['HADM_ID'].isin(III['HADM_ID'])]
    chunk = chunk[chunk['ITEMID'].isin(all_charts)]
    chunk = chunk.dropna(subset=['VALUE', 'VALUENUM'], how='all')
    
    if chunk.shape[0] > 0:
        final = pd.concat([final, chunk], ignore_index=True)
        
final.ITEMID.value_counts()

In [ ]:
print(len(final.HADM_ID.unique()))
print(final[final.VALUENUM.isna()].shape)

In [ ]:
final_match = pd.merge(final,III[['HADM_ID','ICUSTAY_ID','INTIME', 'OUTTIME']],on='HADM_ID',how='left')
print(final_match.shape)

final_match['INTIME'] = pd.to_datetime(final_match['INTIME'])
final_match['OUTTIME'] = pd.to_datetime(final_match['OUTTIME'])
final_match['CHARTTIME'] = pd.to_datetime(final_match['CHARTTIME'])

within_criteria = final_match['CHARTTIME'].between(final_match['INTIME'],final_match['OUTTIME'])
within = final_match[within_criteria]

within['event_time_from_admit'] = within['CHARTTIME'] - within['INTIME']
within['start_time']=within['event_time_from_admit'].dt.total_seconds() / 3600
within['los'] = (within['OUTTIME'] - within['INTIME']).dt.total_seconds() / 3600
within=within[within['start_time']>=0]
print(within.shape)

within_48 = within[within['start_time']<=48]
print(within_48.shape)

within_48['is_numeric'] = within_48['VALUENUM'].apply(is_numeric)

result = within_48.groupby('ITEMID')['is_numeric'].all().reset_index()
result.rename(columns={'is_numeric': 'is_fully_numeric'}, inplace=True)

num_col = result[result.is_fully_numeric==True].ITEMID.values
print(len(within_48.ITEMID.value_counts()),len(num_col))

within_48_make = within_48[['ICUSTAY_ID','ITEMID','VALUENUM']]
agg_df = within_48_make.groupby(['ICUSTAY_ID', 'ITEMID'])['VALUENUM'].agg(
    min='min', 
    max='max', 
    mean='mean', 
    median='median'
).reset_index()

print(agg_df.head())

agg_df =agg_df.drop_duplicates()
matching_columns = list(c_dict.keys())

agg_df['ITEMID'] = agg_df['ITEMID'].astype(str)
agg_df = agg_df[agg_df['ITEMID'].isin(matching_columns)]
agg_df['ITEMID'] = agg_df['ITEMID'].map(c_dict)
print(agg_df.head())

print(len(agg_df.ITEMID.unique()))
print(agg_df.ITEMID.value_counts())

In [ ]:
pivot_lab = agg_df.pivot_table(
    index='ICUSTAY_ID', 
    columns='ITEMID', 
    values=['min', 'max', 'mean', 'median'],
    aggfunc='first'  
)

pivot_lab.columns = [f'{col[1]}_{col[0]}' for col in pivot_lab.columns]
pivot_lab = pivot_lab.reset_index()

pivot_lab = pivot_lab.fillna(pivot_lab.median())
print(pivot_lab.columns.tolist())
pivot_lab.head()

In [ ]:
final_ts = pd.merge(III[['ICUSTAY_ID']],pivot_df,on='ICUSTAY_ID',how='left')
final_ts = pd.merge(final_ts,pivot_lab,on='ICUSTAY_ID',how='left')

In [ ]:
final_ts = final_ts[~final_ts[final_ts.columns.difference(['ICUSTAY_ID'])].isna().all(axis=1)]

In [ ]:
final_ts.shape

In [ ]:
final_ts = final_ts.fillna(final_ts.median())

In [ ]:
Final_III = pd.merge(III,final_ts,on='ICUSTAY_ID',how='left')

In [ ]:
Final_III = Final_III.dropna()

In [ ]:
Final_III.to_csv('ouput/Final_III.csv',index=False)